In [1]:
import os
import pandas as pd
import numpy as np

In [ ]:
569, 665,1283, 1315, 1347, 1379, 1411, 665,1347,1411, 569,601,633, 665,697, 1155,1187, 1219, 1251,1283, 1315, 1347, 1379, 1411,
665, 1283, 1315,1347, 1379, 1411, 

[
[
[
[ 
[
[ 
[ [
[ 
[
[
[
[
[ [
[
[
[
[ 
[
[[
[
[
[



In [ ]:
from google.colab import drive 
drive.mount('/mntDrive') 

Mounted at /mntDrive


In [ ]:

%cd /mntDrive/My Drive/Test_data/test

/mntDrive/My Drive/Test_data/test


In [ ]:
%pwd

'/mntDrive/My Drive/Test_data/test'

In [ ]:
def make_dictionaries(entities, relations, an):
    

    STANDOFF_ENTITY_PREFIX = 'T'
    STANDOFF_RELATION_PREFIX = 'R'
    
    lines = an.readlines()
    for line in lines:
        standoff_line = line.split("\t")
        
        if standoff_line[0][0] == STANDOFF_ENTITY_PREFIX:
            standoff_id= standoff_line[0]
            entities[standoff_id]= {}
            entities[standoff_id]['standoff_id'] = standoff_line[0]
            entities[standoff_id]['entity_type'] =   standoff_line[1].split()[0].upper() 
            if ';' in standoff_line[1]:
                word_1 = standoff_line[1].split(';')[0]
                word_2= standoff_line[1].split(';')[1]
                entities[standoff_id]['offset_start'] = word_1.split()[1]
                entities[standoff_id]['offset_end']= word_2.split()[1]
                entities[standoff_id]['word'] = standoff_line[2].strip('\n')
            else:
                entities[standoff_id]['offset_start'] = standoff_line[1].split()[1]
                entities[standoff_id]['offset_end'] = standoff_line[1].split()[2]
                entities[standoff_id]['word'] = standoff_line[2].strip('\n')
                
        elif standoff_line[0][0] == STANDOFF_RELATION_PREFIX:
            relation_id=  standoff_line[0]
            relations[relation_id]= {}
            relations[relation_id]['standoff_id'] = standoff_line[0]
            splt_str= standoff_line[1].split()
            relations[relation_id]['name'] = splt_str[0]
            relations[relation_id]['standoff_entity1_id'] = splt_str[1].split(':')[1]
            relations[relation_id]['standoff_entity2_id'] = splt_str[2].split(':')[1]
        
            
        an.close()
    return entities, relations

The replace_text and subsequent functions, dataframes were changed for Reason Drug accordingly

In [ ]:
def replace_text_n(text, entities):

    text_n= text
    for ind in range(len(entities)):
        
            if entities[ind]['entity_type']== 'DRUG':
                strt= int(entities[ind]['offset_start'])
                end= int(entities[ind]['offset_end'])
                text_n= text_n[:strt] +'<' + 'e1'+'>'+ entities[ind]['word'] + '</'+ 'e1'+ '>'+ text_n[end:]
            
            if entities[ind]['entity_type']=='ADE':
                strt= int(entities[ind]['offset_start'])
                end= int(entities[ind]['offset_end'])
                text_n= text_n[:strt] +'<' + 'e2'+'>'+ entities[ind]['word'] + '</'+ 'e2'+ '>'+ text_n[end:]
            
                
    return text_n

In [ ]:
def conv_sent(text):
    sents = [sent for sent in text.split('.')]
    
    df= pd.DataFrame(data= sents, columns=['Text'])
    return df

In [ ]:
# df_pos= pd.DataFrame()
# df_neg= pd.DataFrame()
pos_sent_sp_t=[]
neg_sent_sp_t=[]

def positive_rel(sent):
    if w_1 in sent and w_2 in sent:
        pos_sent_sp_t.append(sent)
    elif '<e1>' not in sent and '<e2>' in sent:
        neg_sent_sp_t.append(sent)
    elif '<e1>' in sent and '<e2>' not in sent:
        neg_sent_sp_t.append(sent)
        





ann_data_files = [f for f in os.listdir() if f.split('.')[1] == 'ann']
for file in ann_data_files:
    df_neg_prt= pd.DataFrame()
    
    
    an= open(file, 'r')
    
    
    entities={}
    relations={}
    
    #extract the entities and relations from annotation files
    entities, relations= make_dictionaries(entities, relations, an)
    # sort entities according to the start index of entities.
    ls=[]
    for key in entities.keys():
        ls.append(entities[key])
    sort_ent= sorted(ls, key = lambda i: int(i['offset_start']), reverse= True)
    
    with open(file.replace('.ann', '.txt'), 'r') as document_text_file:
        doc_text= document_text_file.read()
    
   
    
    text_ls= replace_text_n(doc_text, sort_ent) ## replaces the words with entities
    
    df_t= conv_sent(text_ls) ## converts the list into a dataframe
    
    
#neg_sent
    #label='not found'
    for idx in relations:
#     print(idx)
        if relations[idx]['name']== 'ADE-Drug':
            id_1= relations[idx]['standoff_entity1_id']
            id_2= relations[idx]['standoff_entity2_id']
#             print(entities[id_1]['entity_type'])
            if entities[id_1]['entity_type']=='DRUG':
                 w_1='<e1>'+ entities[id_1]['word'] + '</e1>'
            elif entities[id_1]['entity_type']=='ADE':
                 w_1='<e2>'+ entities[id_1]['word']+ '</e2>'
            if entities[id_2]['entity_type']=='ADE':
                 w_2='<e2>'+ entities[id_2]['word'] + '</e2>'
            elif entities[id_2]['entity_type']=='DRUG':
                 w_2='<e1>'+ entities[id_2]['word'] + '</e1>'
#             print(id_1, entities[id_2]['entity_type'])
#             print('The first word is', w_1)
#             print("The second word is", w_2)
            
#             w1= entities[id_1]['word']
#             w2= entities[id_2]['word']
            
            df_t['Text'].apply(positive_rel)

In [ ]:
neg_sent_sp[250]

' <e1>acetaminophen</e1> 325 mg Tablet Sig: 1-2 Tablets PO Q4H (every 4\nhours) as needed for pain/temp'

In [ ]:
ann_data_files = [f for f in os.listdir() if f.split('.')[1] == 'ann']
cnt=0
anot_df= pd.DataFrame()
for file in ann_data_files:
    an= open(file, 'r')
    df_prt = pd.read_csv(an, sep='\t',  error_bad_lines=False, header=None, names=['Index','Details','Word'], engine='python')
    anot_df= anot_df.append(df_prt, ignore_index= False)
    
    an.close()

Skipping line 146: '	' expected after '"'
Skipping line 493: Expected 3 fields in line 493, saw 4
Skipping line 495: Expected 3 fields in line 495, saw 4


In [ ]:
#df_neg= pd.DataFrame(data= neg_sent_sp, columns=['Text'])
df_pos= pd.DataFrame(data= pos_sent_sp, columns=['Text'])

In [ ]:
relations_df = anot_df[anot_df['Index'].str.startswith('R')]
#relations_df.head()

entity_df = anot_df[~anot_df['Index'].str.startswith('R')]


In [ ]:
entity_df.dropna(axis=0, inplace= True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
ade_df= entity_df[entity_df['Details'].str.startswith('A')]
ls_ad= ade_df['Word'].unique()
dr_df= entity_df[entity_df['Details'].str.startswith('Dr')]
ls_dru= dr_df['Word'].unique()

In [ ]:
ade_mp={}
for w1 in ls_ad:
    
    string_1= '<n2>'+ '<e2>' + w1 + '</e2>'
    ade_mp[w1]= string_1

dr_mp={}
for w2 in ls_dru:    
    
    string_2= '<n1>'+ '<e1>' + w2 + '</e1>'
    dr_mp[w2]= string_2

In [ ]:
def extrc_ade(col):
    if '<e2>' in col:
      ex_ade_t.append(col)

def extrc_dr(col):
    if '<e1>' in col:
      ex_drug_t.append(col)

ex_ade_t=[]
ex_drug_t=[]

df_negt_prt= pd.DataFrame(data= neg_sent_sp_t, columns=['Text'])
df_negt_prt['Text'].apply(extrc_ade)
df_negt_prt['Text'].apply(extrc_dr)

In [ ]:
neg_ade_t=[]
for sent_te in ex_drug_t:
  for i, j in ade_mp.items():
    if i in sent_te:
      # print(sent)
      # print(neg_sent_sp.index(sent))
      sent_te1= sent_te
      sent_te1 = re.sub(r"\b%s\b" % i, j, sent_te1)
      
      if j in sent_te1 and sent_te1 not in neg_ade_t:
        neg_ade_t.append(sent_te1)






In [ ]:
neg_ade_t1=[]
for sent_t in ex_ade_t:
  for i, j in dr_mp.items():
    if i in sent_t:
      #print(sent)
      # print(neg_sent_sp.index(sent))
      sent_t1= sent_t
      sent_t1 = re.sub(r"\b%s\b" % i, j, sent_t1)
      
      if j in sent_t1 and sent_t1 not in neg_ade_t1:
        neg_ade_t1.append(sent_t1)

In [ ]:
df_pos_t= pd.DataFrame(data= pos_sent_sp_t, columns=['Text'])


In [ ]:
df_neg_adet= pd.DataFrame(data= neg_ade_t, columns=['Text'])


In [ ]:
df_neg_adet['Text']= df_neg_ade['Text'].str.replace('<n1>', ' ')
df_neg_adet['Text']= df_neg_ade['Text'].str.replace('<n2>', ' ')
df_neg_adet['Text'].head()

0     for breast cancer\ndiagnosed in [**11/2098**]...
1     for breast cancer diagnosed in\n[**11/2098**]...
2    \n# Acute  <e2>renal failure</e2>:\n- <e1>IVFs...
3    \n#  <e2>Acute renal failure</e2>:\n- <e1>IVFs...
4    5 mg [**Hospital1 **] prn\n <e2>anxiety</e2>\n...
Name: Text, dtype: object

In [ ]:
#df_pos_t= pd.DataFrame(data= pos_sent_sp_t, columns=['Text'])
df_pos_t['relation']= 'ADE-DRUG(e2-e1)'

In [ ]:
df_neg_adet['relation']='Other'

In [ ]:
print(neg_ade_t[710], '\n')
print(neg_ade_t[10])
print(neg_ade_t[15])

 At that same time he was started on
<e1>Vancomycin</e1> since his <n2><e2>allergy</e2> was questionable and he had a
permanent tunneled line 

  His post-operative course was
complicated by <n2><e2>fevers</e2>, confusion, unstable vitals consistent
with urosepsis/shock, requiring transfer to the  MICU and broad
<e1>antibiotic</e1> therapy, now transferred back to the [**Doctor Last Name 3271**]-[**Doctor Last Name 679**]
service, stable, and ready for discharge

 You have now completed intravenous <e1>antibiotic</e1> therapy for that
<n2><e2>infection</e2> and have three days remaining of an oral <e1>antibiotic</e1>
(see below)


In [ ]:
df_ade_t= pd.concat([df_pos_t, df_neg_adet], axis=0, ignore_index= True)


In [ ]:
df_ade_t.to_csv(r'/mntDrive/My Drive/Edge Sampling/ade_test_new_edge.csv', encoding='utf-8')